In [16]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_and_chunk_pdf(path):
    reader = PdfReader(path)
    raw_text = ""

    for page in reader.pages:
        if page.extract_text():
            raw_text += page.extract_text()

    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = splitter.split_text(raw_text)

    return chunks


In [17]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document

def create_faiss_index(chunks):
    docs = [Document(page_content=chunk) for chunk in chunks]

    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    db = FAISS.from_documents(docs, embeddings)
    return db



In [27]:
def retrieve_docs(query, db, k=15):
    results = db.similarity_search(query, k=k)
    return results


In [28]:
import streamlit as st
import requests

import requests

GROQ_API_KEY = "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"  

def generate_answer_groq(query, context, model="llama3-8b-8192"):
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }

    system_prompt = "You are a helpful assistant that answers questions based only on the provided context."
    user_prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"

    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "temperature": 0.2,
        "max_tokens": 300
    }

    response = requests.post(url, headers=headers, json=payload)

    try:
        data = response.json()
        if "choices" in data:
            return data["choices"][0]["message"]["content"]
        else:
            return f"⚠️ Unexpected response: {data}"
    except Exception as e:
        return f"❌ Error: {e} | Raw response: {response.text}"



In [20]:
def rag_chatbot(query, db):
    docs = retrieve_docs(query, db)
    context = "\n\n".join([doc.page_content for doc in docs])
    answer = generate_answer_groq(query, context)
    return answer


In [29]:
chunks = load_and_chunk_pdf("/Users/pmanthan/Desktop/AICommunity_Assignment_25.pdf")
db = create_faiss_index(chunks)

response = rag_chatbot("what is the second technical question in detail",db)
print(response)

The second technical question is:

1.2 Q2: Transfer Learning for Fashion-MNIST

Objective: Adapt a pretrained CNN (e.g., ResNet50, VGG16) to classify 28 ×28 grayscale Fashion-MNIST images into 10 classes.

Implementation:

a) Data pipeline:

* Resize to 224 ×224.
* Convert 1 →3 channels (duplication or learnable adapter).

b) Model:

* Load pretrained backbone without top layers.
* Freeze backbone; add new FC head.
* Train head only; record validation metrics.

c) Fine-tuning:

* Unfreeze selected deeper blocks.

Note: The goal is to adapt a pre-trained CNN to classify Fashion-MNIST images, using transfer learning. The implementation involves resizing the images, converting the channels, loading the pre-trained backbone, adding a new FC head, and training the head only. The fine-tuning step involves unfreezing selected deeper blocks.
